In [20]:
import openeo

In [21]:
connection = openeo.connect("openeofed.dataspace.copernicus.eu")
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeofed.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

In [22]:
c_ids = filter(lambda x: x.startswith("SENTINEL1"), connection.list_collection_ids())
for c_id in c_ids:
    print(c_id)

SENTINEL1_GRD
SENTINEL1_GLOBAL_MOSAICS


In [23]:
long_start = 3.5
long_end = 8.5
long_stride = 1

lat_start = 51.5
lat_end = 55.5
lat_stride = 1

In [24]:
import math
from datetime import datetime

long_steps = [long_start + i * long_stride for i in range(math.ceil((long_end - long_start) / long_stride))]
lat_steps = [lat_start + i * lat_stride for i in range(math.ceil((lat_end - lat_start) / lat_stride))]


In [25]:
for lat_step_start in lat_steps:
    lat_step_end = lat_step_start + lat_stride
    for long_step_start in long_steps:
        long_step_end = long_step_start + long_stride

        bbox = {"west": long_step_start, "south": lat_step_start, "east": long_step_end, "north": lat_step_end}
        print(f"downloading {bbox}")

        datacube = connection.load_collection(
            "SENTINEL1_GRD",
            temporal_extent = "2024-05",
            spatial_extent = bbox,
            bands = ["VV","VH"],
        )

        data_slice = datacube.reduce_temporal("mean")

        file_slug = f"../data/sentinel1_grd/2024-05_{lat_step_start}-{lat_step_end}_{long_step_start}-{long_step_end}"
        data_slice.band("VV").download(file_slug + "_vv.tif")
        data_slice.band("VH").download(file_slug + "_vh.tif")
        combined_band = data_slice.reduce_bands("mean")
        combined_band.download(file_slug + "_vv&vh.tif")


downloading {'west': 3.5, 'south': 51.5, 'east': 4.5, 'north': 52.5}


KeyboardInterrupt: 